# Demo Notebook

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from datasets import load_dataset

/home/technet/miniconda3/envs/technet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset('snli')

Generating train split: 100%|█████████████████████████████████████████████████████████████████████████████| 550152/550152 [00:00<00:00, 1785507.92 examples/s]


In [9]:
dataset['train'].to_pandas()

,premise,hypothesis,label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,1
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",2
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",0
3,Children smiling and waving at camera,They are smiling at their parents,1
4,Children smiling and waving at camera,There are children present,0
...,...,...,...
550147,Four dirty and barefooted children.,four kids won awards for 'cleanest feet',2
550148,Four dirty and barefooted children.,"four homeless children had their shoes stolen,...",1
550149,A man is surfing in a bodysuit in beautiful bl...,A man in a bodysuit is competing in a surfing ...,1
550150,A man is surfing in a bodysuit in beautiful bl...,A man in a business suit is heading to a board...,2


In [8]:
dataset['validation'].to_pandas()

,premise,hypothesis,label
0,Two women are embracing while holding to go pa...,The sisters are hugging goodbye while holding ...,1
1,Two women are embracing while holding to go pa...,Two woman are holding packages.,0
2,Two women are embracing while holding to go pa...,The men are fighting outside a deli.,2
3,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.,0
4,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.,1
...,...,...,...
9995,A small girl wearing a pink jacket is riding o...,The girl is sitting on a carved horse made of ...,-1
9996,A small girl wearing a pink jacket is riding o...,The girl is moving at the speed of light.,2
9997,A young girl with blue and pink ribbons in her...,People in a water fountain,0
9998,A young girl with blue and pink ribbons in her...,A young girl knits a sweater,2


In [2]:
from __future__ import absolute_import, division, unicode_literals

import sys
import numpy as np
import logging
import sklearn
import torch

# Set PATHs
# path to senteval
PATH_TO_SENTEVAL = 'SentEval/'
# path to the NLP datasets 
PATH_TO_DATA = 'SentEval/data/'
# path to glove embeddings
PATH_TO_VEC = 'pretrained/glove.840B.300d.txt'

# import SentEval
sys.path.insert(0, PATH_TO_SENTEVAL)
import senteval
from encoders.word_embeddings_mean import WordEmbeddingsMeanEncoder
from utils.word_embeddings import create_dictionary, get_wordvec

In [3]:
sentences = [['hello', 'it', 'is', 'your', 'mother'], ['i', 'really', 'like', 'your', 'mothers', 'pancake']]

In [4]:
id2word, word2id = create_dictionary(sentences)

In [5]:
word2vec = get_wordvec(PATH_TO_VEC, word2id)

In [6]:
word2vec['is']

array([-8.4961e-02,  5.0200e-01,  2.3823e-03, -1.6755e-01,  3.0721e-01,
       -2.3762e-01,  1.6069e-01, -3.6786e-01, -5.8347e-02,  2.4990e+00,
       -2.3647e-03,  1.0732e-02, -3.0422e-01,  8.4579e-02, -4.0299e-02,
       -4.1562e-01, -2.4494e-02,  1.4691e+00, -5.2932e-02, -7.4413e-02,
       -3.9244e-01, -3.2535e-01, -2.2333e-01,  5.6823e-03,  3.5675e-01,
        1.9445e-01,  5.6762e-02, -4.5502e-02, -2.8105e-01, -5.8896e-02,
       -9.8626e-02,  9.2177e-02,  3.3172e-01, -3.9967e-02, -1.1766e-01,
        4.8373e-02, -6.2241e-02, -1.0413e-01,  9.9263e-04, -4.8925e-01,
        3.4786e-01,  3.2724e-01,  1.3882e-01, -1.9917e-01,  1.2995e-01,
        6.0549e-02, -2.3714e-01, -5.1295e-01, -3.7396e-01,  1.2902e-01,
        5.5797e-02,  3.3444e-01, -1.8025e-01, -3.4740e-02,  2.8323e-01,
       -9.5301e-02,  2.1143e-01, -7.6149e-02,  1.5069e-01, -1.7441e-01,
       -7.4768e-03, -7.8287e-02, -1.2751e-01,  2.2545e-01,  3.5101e-02,
       -6.1015e-01, -2.6812e-01,  6.1632e-02, -3.0503e-01, -1.34

# 1. Run SentEval

In [103]:
from encoders.lstm.unidirectional_lstm import UnidirectionalLSTMEncoder
from encoders.lstm.bidirectional_lstm import BidirectionalLSTMEncoder

In [118]:
sentence_encoder = BidirectionalLSTMEncoder(pooling_type='max') # UnidirectionalLSTMEncoder() # WordEmbeddingsMeanEncoder()

In [119]:
test_sentence = ['hello', 'it', 'is', 'your', 'mother']

In [120]:
embeddings = torch.Tensor([[word2vec[word] for word in test_sentence]])

In [121]:
embeddings.shape

torch.Size([1, 5, 300])

In [122]:
sentence_encoder.forward((embeddings, [5]))[0].detach().numpy().shape

(4096,)

In [109]:
sentence_encoder.forward((embeddings, [5])).shape

torch.Size([1, 4096])

In [110]:
from utils.word_embeddings import get_word_embeddings

In [52]:
# SentEval prepare and batcher
def prepare(params, samples):
    _, params.word2id = create_dictionary(samples)
    params.word_vec = get_wordvec(PATH_TO_VEC, params.word2id)
    params.wvec_dim = 300
    return

def batcher(params, batch):
    batch = [sent if sent != [] else ['.'] for sent in batch]
    embeddings = []

    for sent in batch:
        word_embeddings = get_word_embeddings(params.word_vec, sent)
        if isinstance(sentence_encoder, WordEmbeddingsMeanEncoder):
            sentvec = sentence_encoder.forward(torch.Tensor([word_embeddings]))[0].detach().numpy()
        else:
            sentvec = sentence_encoder.forward((torch.Tensor([word_embeddings]), [len(sent)]))[0].detach().numpy()
        embeddings.append(sentvec)

    embeddings = np.vstack(embeddings)
    return embeddings

In [53]:
# Set params for SentEval
# we use logistic regression (usepytorch: Fasle) and kfold 10
# In this dictionary you can add extra information that you model needs for initialization
# for example the path to a dictionary of indices, of hyper parameters
# this dictionary is passed to the batched and the prepare fucntions
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': False, 'kfold': 4}
# this is the config for the NN classifier but we are going to use scikit-learn logistic regression with 10 kfold
# usepytorch = False 
# params_senteval['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
#                                 'tenacity': 3, 'epoch_size': 2}

# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

se = senteval.engine.SE(params_senteval, batcher, prepare)

# here you define the NLP taks that your embedding model is going to be evaluated
# in (https://arxiv.org/abs/1802.05883) we use the following :
# SICKRelatedness (Sick-R) needs torch cuda to work (even when using logistic regression), 
# but STS14 (semantic textual similarity) is a similar type of semantic task
transfer_tasks = ['MRPC']
# senteval prints the results and returns a dictionary with the scores
results = se.eval(transfer_tasks)
print(results)

KeyboardInterrupt: 

In [ ]:
results # WordMean: 0.746 accuracy

# 2. Run SNLI

In [54]:
from utils.snli_data import preprocess_text

[nltk_data] Downloading package punkt to /home/technet/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [55]:
%%time
preprocess_text("""hello what's up? I miss you.""")

CPU times: user 8.93 ms, sys: 0 ns, total: 8.93 ms
Wall time: 9.09 ms


['hello', 'whats', 'up', 'i', 'miss', 'you']

# Train on SNLI

In [56]:
import torch.nn as nn
import torch

In [57]:
from utils.snli_data import get_snli_data
from heads.snli_model import SNLIClassifier

In [58]:
train = get_snli_data(split='train', sample=20_000)
valid = get_snli_data(split='dev')
test = get_snli_data(split='test')

reading train
sampling...
tokenizing sentence 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:01<00:00, 13154.55it/s]


tokenizing sentence 2


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:01<00:00, 16971.33it/s]


reading dev
tokenizing sentence 1


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 16647.49it/s]


reading test
tokenizing sentence 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 12932.83it/s]


tokenizing sentence 2


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 16143.49it/s]


In [59]:
all_sentences = list(train['sentence1']) + list(train['sentence2']) +\
                list(valid['sentence1']) + list(valid['sentence2']) +\
                list(test['sentence1']) + list(test['sentence2'])

In [60]:
id2word, word2id = create_dictionary(all_sentences)

In [61]:
def get_longest_sentence(sentences):
    return np.max([len(s) for s in sentences])

In [62]:
MAX_LEN = get_longest_sentence(all_sentences)
MAX_LEN

64

In [111]:
word2vec = get_wordvec(PATH_TO_VEC, word2id)

In [117]:
nli_model = SNLIClassifier(encoder=BidirectionalLSTMEncoder(encoding_lstm_dim=256, pooling_type='max'), embedding_dim=512)

In [123]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=nli_model.parameters(), lr=0.1)

In [124]:
BATCH_SIZE = 64

In [125]:
from train_snli import train_one_epoch
from utils.eval import evaluate_model

In [126]:
for epoch in range(10):
    print('epoch', epoch)
    loss_batches = train_one_epoch(nli_model, train, optimizer, loss_fn, word2vec, BATCH_SIZE, tuple_input=True)
    print(np.mean(loss_batches))
    print(evaluate_model(nli_model, valid, word2vec, BATCH_SIZE, tuple_input=True))

epoch 0


313it [02:18,  2.26it/s]                                                                                                                                            


1.0934857518528216


154it [00:13, 11.08it/s]                                                                                                                                            


{'accuracy': 0.3330623856939646}
epoch 1


...

0.6746816810922691


154it [00:10, 14.62it/s]                                                                                                                                            


{'accuracy': 0.6588091851249746}
epoch 5


 24%|██████████████████████████████▎                                                                                               | 75/312 [00:30<01:36,  2.47it/s]


KeyboardInterrupt: 

In [ ]:
X